In [ ]:
# Imports
import gymnasium as gym
from typing import Callable

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch as th

import pickle

# Import Our environment
from dev_env import tradingEng


c:\Users\osc16\miniconda3\envs\d2d\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [ ]:
# Load Paths
with open("1.6kRunDemo.pkl","rb") as fp:
    paths = pickle.load(fp)

In [ ]:
## LR schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [ ]:
# Def Env
env = tradingEng(paths, action = 'big', obs = 'big-nt')

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[512,512,512,512,256,128,64,64,64,64,64,36,18], vf=[512,512,512,512,256,128,64,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = False, ortho_init = False))
model = PPO("MlpPolicy", env, verbose=True, batch_size = 1200*5, learning_rate=linear_schedule(5e-4), policy_kwargs=policy_kwargs, n_steps=1200*10, normalize_advantage=True, vf_coef = 0.9, gamma = 0.9,ent_coef=0.01)

model.learn(total_timesteps=2e7, log_interval=10)
# Save the agent
model.save("PPOBig")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


ValueError: could not broadcast input array from shape (36,) into shape (37,)

In [14]:
import numpy as np
print(env._get_obs())
path = paths[800]
lambdax = path.lambdas[25]
r = path.r[25]
t = path.t_s[25]
print(lambdax)
print(r)

#model.predict(observation=np.asmatrix([lambdax,r,t]),state=None, deterministic=True)

[[-0.99977158  0.05536838 -0.56841282]]
0.005196713067477058
0.030745992173113156
